# CMR Stac ➡ DuckDB
Based on `stac_geoparquet.ipynb` and `duck_machine.ipynb`

In [40]:
import json
import os
import requests

import duckdb
import pyarrow.parquet as pq
import stac_geoparquet

In [29]:
provider = 'LPCLOUD'
short_name = 'MCD19A2_061'
page_size = 100
max_records = 2000

base_path = f"data/{provider}/{short_name}"
os.makedirs(base_path, exist_ok=True)

## Download pages of STAC Items

For each FeatureCollection response from the STAC Items endpoint, write each of its features into a `jsonl` file. Append to this file for each FeatureCollection.

In [30]:
base = f"https://cmr.earthdata.nasa.gov/stac/{provider}/collections/{short_name}/items?limit={page_size}"
current = base
batch_count = 0

stac_out_basename = f"{short_name}_{max_records}"
stac_out_path = os.path.join(base_path, stac_out_basename + ".jsonl")

with open (stac_out_path, 'w') as jsonl:
  while len(current) > 0:
      batch_count += 1
      resp = requests.get(current)

      # check for end of data
      if resp.status_code!=200:
          message = resp.json()['errors'][0]
          print(f"\033(31mError: {resp.status_code} - {resp.reason}: {message}\n{current}\033(0m")
          print(f"On batch number {batch_count}")
          break # the cursor is broken, get out


      # write items from collection batch as json lines
      for item in data['features']:
        json.dump(item, jsonl, separators=(",", ":"))
        jsonl.write("\n")

      # exit if reached maximum record count
      if page_size * batch_count >= max_records:
        print(f"Reached max_records = {max_records}")
        break
      else:
        print(f"Finished writing lines for {(batch_count - 1) * page_size}-{batch_count * page_size}")

      # look for next page
      data = resp.json()
      for link in data['links']:
          if link['rel']=='next':
              next = link['href']
              current = next if current != next else ''

Finished writing lines for 0-100
Finished writing lines for 100-200
Finished writing lines for 200-300
Finished writing lines for 300-400
Finished writing lines for 400-500
Finished writing lines for 500-600
Finished writing lines for 600-700
Finished writing lines for 700-800
Finished writing lines for 800-900
Finished writing lines for 900-1000
Finished writing lines for 1000-1100
Finished writing lines for 1100-1200
Finished writing lines for 1200-1300
Finished writing lines for 1300-1400
Finished writing lines for 1400-1500
Finished writing lines for 1500-1600
Finished writing lines for 1600-1700
Finished writing lines for 1700-1800
Finished writing lines for 1800-1900
Reached max_records = 2000


## MODIS Sinusoidal Grid

![](https://lpdaac.usgs.gov/media/images/modis_sinusoidal_grid.width-800.jpg)

How many unique MODIS SIN grid tiles (unique geometries) are in this query?

The [maximum unique tiles would be 460](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/modis-overview/#modis-tiling-systems), but not every collection would include data for every tile.

In [39]:
%%bash -s "$stac_out_path"
grep -o -e 'h[01].v[01].' $1 | sort | uniq | wc -l

     163


## Read JSONL output as Arrow table

Then check schema and dimensions

In [31]:
record_batch_reader = stac_geoparquet.arrow.parse_stac_ndjson_to_arrow(stac_out_path)
table = record_batch_reader.read_all()
print("Arrow table shape:   ", table.shape)
print("Arrow table columns: \n  ", str(table.column_names).replace(',', ',\n  '))

Arrow table shape:    (2000, 13)
Arrow table columns: 
   ['assets',
   'bbox',
   'collection',
   'geometry',
   'id',
   'links',
   'stac_extensions',
   'stac_version',
   'type',
   'datetime',
   'end_datetime',
   'eo:cloud_cover',
   'start_datetime']


### Convert from JSONL to Geoparquet

In [32]:
out_parquet_path = os.path.join(base_path, stac_out_basename + ".parquet")
stac_geoparquet.arrow.parse_stac_ndjson_to_parquet(stac_out_path, out_parquet_path)

Confirm Geoparquet output has same structure as Arrow version

In [34]:
pq_table = pq.read_table(out_parquet_path)
print("Parquet table shape:   ", pq_table.shape)
print("Parquet table columns: \n  ", str(pq_table.column_names).replace(',', ',\n  '))

Parquet table shape:    (2000, 13)
Parquet table columns: 
   ['assets',
   'bbox',
   'collection',
   'geometry',
   'id',
   'links',
   'stac_extensions',
   'stac_version',
   'type',
   'datetime',
   'end_datetime',
   'eo:cloud_cover',
   'start_datetime']


### Verify compliance with `gpq`

In [38]:
%%bash -s "$out_parquet_path"
gpq validate $1


Summary: Passed 20 checks.

 ✓ file must include a "geo" metadata key
 ✓ metadata must be a JSON object
 ✓ metadata must include a "version" string
 ✓ metadata must include a "primary_column" string
 ✓ metadata must include a "columns" object
 ✓ column metadata must include the "primary_column" name
 ✓ column metadata must include a valid "encoding" string
 ✓ column metadata must include a "geometry_types" list
 ✓ optional "crs" must be null or a PROJJSON object
 ✓ optional "orientation" must be a valid string
 ✓ optional "edges" must be a valid string
 ✓ optional "bbox" must be an array of 4 or 6 numbers
 ✓ optional "epoch" must be a number
 ✓ geometry columns must not be grouped
 ✓ geometry columns must be stored using the BYTE_ARRAY parquet type
 ✓ geometry columns must be required or optional, not repeated
 ✓ all geometry values match the "encoding" metadata
 ✓ all geometry types must be included in the "geometry_types" metadata (if not empty)
 ✓ all polygon geometries must follow

## Query Geoparquet file with DuckDB

Test point in Canada. Should only match `h11v03`, but matches at least 3 other tiles, too.  
This point is very far (>260km) from any borders of the MODIS grid tile.

In [46]:
duckdb.sql(f'''
SELECT st_contains(geometry::geometry, 'POINT(-107.5 53.3)'::GEOMETRY) as found,
    id,
    datetime
FROM parquet_scan('{out_parquet_path}')
WHERE found == true
''')

┌─────────┬───────────────────────────────────────────┬──────────────────────────┐
│  found  │                    id                     │         datetime         │
│ boolean │                  varchar                  │ timestamp with time zone │
├─────────┼───────────────────────────────────────────┼──────────────────────────┤
│ true    │ MCD19A2.A2000058.h08v03.061.2022153225217 │ 2000-02-26 19:40:00-05   │
│ true    │ MCD19A2.A2000058.h09v03.061.2022153225142 │ 2000-02-26 19:40:00-05   │
│ true    │ MCD19A2.A2000058.h07v03.061.2022153223310 │ 2000-02-26 19:45:00-05   │
│ true    │ MCD19A2.A2000055.h07v03.061.2022153215825 │ 2000-02-23 19:10:00-05   │
│ true    │ MCD19A2.A2000055.h09v03.061.2022153221548 │ 2000-02-23 19:10:00-05   │
│ true    │ MCD19A2.A2000055.h08v03.061.2022153222217 │ 2000-02-23 19:10:00-05   │
│ true    │ MCD19A2.A2000055.h11v03.061.2022153220110 │ 2000-02-24 11:45:00-05   │
│ true    │ MCD19A2.A2000056.h09v03.061.2022153222246 │ 2000-02-24 19:55:00-05   │
│ tr

### OSU Test Point
Test point on the Oval of Ohio State University, Columbus. Should only match `h11v04`, but also matches `h11v05` to the south.  
This point is only 4km north of the border, but I would have expected east-west boundaries to have more potential for inaccurate intersections than north-south ones.

In [ ]:
duckdb.sql(f'''
SELECT st_contains(geometry::geometry, 'POINT(-83.0123 40)'::GEOMETRY) as found,
    id,
    datetime
FROM parquet_scan('{out_parquet_path}')
WHERE found == true
''')

┌─────────┬───────────────────────────────────────────┬──────────────────────────┐
│  found  │                    id                     │         datetime         │
│ boolean │                  varchar                  │ timestamp with time zone │
├─────────┼───────────────────────────────────────────┼──────────────────────────┤
│ true    │ MCD19A2.A2000055.h11v05.061.2022153215223 │ 2000-02-24 10:10:00-05   │
│ true    │ MCD19A2.A2000055.h11v04.061.2022153215336 │ 2000-02-24 11:45:00-05   │
│ true    │ MCD19A2.A2000056.h11v05.061.2022153221329 │ 2000-02-25 10:50:00-05   │
│ true    │ MCD19A2.A2000056.h11v04.061.2022153221449 │ 2000-02-25 10:50:00-05   │
│ true    │ MCD19A2.A2000057.h11v05.061.2022153222247 │ 2000-02-26 10:00:00-05   │
│ true    │ MCD19A2.A2000057.h11v04.061.2022153223331 │ 2000-02-26 11:30:00-05   │
│ true    │ MCD19A2.A2000058.h11v05.061.2022153223828 │ 2000-02-27 10:40:00-05   │
│ true    │ MCD19A2.A2000058.h11v04.061.2022153225015 │ 2000-02-27 10:40:00-05   │
│ tr

## PSU Test Point
Test point at Penn State University's Berkey Creamery. Should only match `h12v04`, and does.  
While this point is relatively near to the southwest corner of the MODIS grid tile, it is still nearly 100km away from it.

In [45]:
duckdb.sql(f'''
SELECT st_contains(geometry::geometry, 'POINT(-77.862 40.803)'::GEOMETRY) as found,
    id,
    datetime
FROM parquet_scan('{out_parquet_path}')
WHERE found == true
''')

┌─────────┬───────────────────────────────────────────┬──────────────────────────┐
│  found  │                    id                     │         datetime         │
│ boolean │                  varchar                  │ timestamp with time zone │
├─────────┼───────────────────────────────────────────┼──────────────────────────┤
│ true    │ MCD19A2.A2000055.h12v04.061.2022153215604 │ 2000-02-24 10:05:00-05   │
│ true    │ MCD19A2.A2000056.h12v04.061.2022153221624 │ 2000-02-25 10:50:00-05   │
│ true    │ MCD19A2.A2000057.h12v04.061.2022153222819 │ 2000-02-26 09:55:00-05   │
│ true    │ MCD19A2.A2000058.h12v04.061.2022153224130 │ 2000-02-27 10:35:00-05   │
│ true    │ MCD19A2.A2000059.h12v04.061.2022153225050 │ 2000-02-28 09:45:00-05   │
│ true    │ MCD19A2.A2000060.h12v04.061.2022153225921 │ 2000-02-29 10:25:00-05   │
└─────────┴───────────────────────────────────────────┴──────────────────────────┘